In [5]:
import open3d as o3d
import numpy as np
from IPython.display import display, HTML

In [6]:
def load_point_cloud(file_path):
    """Load a point cloud from a file."""
    cloud = o3d.io.read_point_cloud(file_path)
    return cloud

def align_point_clouds(source_cloud, target_cloud):
    """Align two point clouds using ICP."""
    icp_transformation = o3d.pipelines.registration.registration_icp(
        source_cloud, target_cloud, max_correspondence_distance=25,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000)
    )

    aligned_source_cloud = source_cloud.transform(icp_transformation.transformation)
    return aligned_source_cloud

def save_point_cloud(file_path, point_cloud):
    """Save a point cloud to a file."""
    o3d.io.write_point_cloud(file_path, point_cloud)

def visualize_pointcloud(pointcloud):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pointcloud)

    # Capture the generated image and display it in the notebook
    image = vis.capture_screen_float_buffer(True)
    display(o3d.visualization.draw_geometries_with_animation_callback([pointcloud], vis, interval=50))
    
    # Clean up resources after visualization
    vis.destroy_window()

def downsample_pointcloud(pointcloud, voxel_size):
    # Voxel downsample the point cloud
    downsampled_pc = pointcloud.voxel_down_sample(voxel_size)
    return downsampled_pc
    

In [7]:
ply_path = "../output/bim.ply"
obj_path = "../output/pcfull.ply"

source_cloud = load_point_cloud(ply_path)
target_cloud = load_point_cloud(obj_path)

In [8]:
o3d.visualization.draw_geometries([source_cloud ])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
downsampled_pointcloud = downsample_pointcloud(source_cloud , voxel_size=0.005)
visualize_pointcloud(downsampled_pointcloud)

In [4]:
visualizer = JVisualizer()
visualizer.add_geometry(source_cloud)
visualizer.show()

NameError: name 'JVisualizer' is not defined

In [ ]:
source_cloud = np.asarray(load_point_cloud(ply_path).points).astype(np.float32)
target_cloud = np.asarray(load_point_cloud(obj_path).points).astype(np.float32)

In [ ]:
source_center = np.mean(source_cloud, axis=0)
print(source_center)
target_center = np.mean(target_cloud, axis=0)
print(target_center)
diff = target_center - source_center
print(diff)
source_cloud = source_cloud + diff
print(np.mean(source_cloud))